# Gene ID mapping

![](./images/Module1/Gene_ID_Conversion.png)
*(Source: david.ncifcrf.gov)*

## Learning Objectives:
1. Understanding different probe set ID.
2. Mapping of various protein/gene IDs into gene identifiers and symbols.

### Understanding different probe set ID

Gene set or pathway analysis requires that gene sets and expression data use the same type of gene ID (Entrez Gene ID, Gene symbol or probe set ID etc). However, this is frequently not true for the data we have. For example, our gene sets mostly use Entrez Gene ID, but microarray datasets are frequently labeled by probe set ID (or RefSeq transcript ID etc). Therefore, we need to convert or map the probe set IDs to Entrez gene ID. Here, we will use `GSE48350` dataset that we have used in the previous section for demonstration of gene ID mapping. In order to know what kind of probe set ID, we need to navigate to GEO record page of `GSE48350`. Under the `Platform` tab, we can find the probe ID information.
![probID](./images/Module1/ProbeID.png)

From the record page, we know that the dataset was generated from 1 platform using Affymetrix Human Genome U133 Plus 2.0 Array. To convert or map the probe set IDs to Entrez gene ID, we need to find the corresponding annotation package from <a href="https://bioconductor.org/">Bioconductor</a>. For analyzed data, we need to use the `hgu133plus2.db` database. We can install the package using following R command:

In [1]:
#if (!require("BiocManager", quietly = TRUE)
#   install.packages("BiocManager")
BiocManager::install("hgu133plus2.db", update = F)

'getOption("repos")' replaces Bioconductor standard repositories, see
'?repositories' for details

replacement repositories:
    CRAN: https://cran.r-project.org




Bioconductor version 3.14 (BiocManager 1.30.16), R 4.1.3 (2022-03-10)



Warning message:
"package(s) not installed when version(s) same as current; use `force = TRUE` to
  re-install: 'hgu133plus2.db'"


In the previous section, we have successfully downloaded the dataset `GSE48350` and saved a sub-set of it to the local storage. Now we can load the sample dataset and check for the Probe Set ID names by using the following command.

In [2]:
data = readRDS("./data/GSE48350.rds")
probeIDs = rownames(data)
probeIDs

[1] "1007_s_at" "1053_at"   "117_at"    "121_at"    "1255_g_at" "1294_at"  
 [7] "1316_at"   "1320_at"   "1405_i_at" "1431_at"

**Probe Set ID** is the identifier that refers to a set of probe pairs selected to represent expressed sequences on an array. Designations are given at design time.

The probe set names never change, but they can give you an idea of what was known about the sequence at the time of design.

* **_at** = all the probes hit one known transcript.
* **_a** = all probes in the set hit alternate transcripts from the same genes.
* **_s** = all probes in the set hit transcripts from different genes.
* **_x** = some probes hit transcripts from different genes.

For HG-U133, the **_a** designation was not used; an **_s** probe set on these arrays means the same as an **_a** on any of the HG-U133 arrays.

### Mapping of various protein/gene IDs into gene identifiers and symbols.
For pathway analysis, Probe set IDs should be mapped to Ensembl ID and SYMBOL. Ensembl ID is a transcript identifier from the ENSEMBL project while SYMBOL is a gene symbol is a short-form abbreviation for a particular gene. We can load the `hgu133plus2.db` database to perform Probe ID mapping using the following command:

In [3]:
library(hgu133plus2.db)

Loading required package: AnnotationDbi



Loading required package: stats4



Loading required package: BiocGenerics




Attaching package: 'BiocGenerics'




The following objects are masked from 'package:stats':

    IQR, mad, sd, var, xtabs




...


Attaching package: 'S4Vectors'




The following objects are masked from 'package:base':

    expand.grid, I, unname





Attaching package: 'IRanges'




The following object is masked from 'package:grDevices':

    windows




Loading required package: org.Hs.eg.db



Here, we can build a  lookup table that maps the Probe ID to Ensembl ID and gene symbol using the following command:

In [4]:
annotLookup <- select(hgu133plus2.db, keys = probeIDs, columns = c('PROBEID', 'ENSEMBL', 'SYMBOL'))

'select()' returned 1:many mapping between keys and columns



In [5]:
annotLookup

PROBEID,ENSEMBL,SYMBOL
<chr>,<chr>,<chr>
1007_s_at,ENSG00000204580,DDR1
1007_s_at,ENSG00000223680,DDR1
1007_s_at,ENSG00000229767,DDR1
1007_s_at,ENSG00000234078,DDR1
1007_s_at,ENSG00000215522,DDR1
1007_s_at,ENSG00000137332,DDR1
1007_s_at,ENSG00000230456,DDR1
1053_at,ENSG00000049541,RFC2
117_at,ENSG00000173110,HSPA6


From the lookup table we can spot that a single Probe ID can be mapped to multiple Ensembl IDs. Depending on analysis performed, users can select their own Ensembl ID of interest. Here is a simple way to replace Probe IDs with gene symbols :

In [6]:
rownames(data) <- unique(annotLookup$SYMBOL)

In [7]:
data


,GSM300166,GSM300167,GSM300168,GSM300169,GSM300170,GSM300171,GSM300172,GSM300173,GSM300174,GSM300175
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
DDR1,-0.17134216,0.52157192,0.68670355,0.81143831,0.86451700,0.52574016,0.66688491,-0.93040715,-0.328126840,-0.74640518
RFC2,-0.58540894,-0.17486225,0.89458707,0.05195171,0.50730550,0.57049732,0.24645953,-0.18996975,-0.123880381,0.02598866
HSPA6,-0.21819817,0.08682425,1.63105514,1.16585098,2.33922329,3.78668177,1.14488628,-0.27841910,0.723029702,-0.46762850
PAX8,-0.03630468,0.07141232,-0.02578819,0.26091293,0.05593042,-0.35576714,0.01708155,0.20353308,-0.058768961,-0.13781851
GUCA1A,-1.02545683,-0.89559517,1.66886552,0.59864181,1.12816206,1.82441916,1.14608539,0.50837510,2.325217438,0.41166042
UBA7,-0.26114260,-0.10595602,0.60347267,0.63885094,0.71480600,0.03780249,1.20230186,-0.35426747,0.007652498,-0.18228540
THRA,0.72345948,0.01939904,-0.78443074,-0.71589564,-0.46461587,-0.98535048,-0.19650176,0.37064082,-0.316624769,-0.06715241
PTPN21,-0.10373373,-0.38622528,-1.00999569,-1.13029217,-1.37365271,-1.08647842,-0.89696490,-0.39420333,0.101918960,-0.13101714
CCL5,0.64007049,-0.27436622,0.80047113,0.21786040,-0.14264812,0.11026261,1.04344810,0.08017696,0.202078644,-0.33860081
